# Dimensions, Facets & Animation

How to control which dimensions map to which visual properties.

In [ ]:
import plotly.express as px
import xarray as xr

from xarray_plotly import config, xpx

config.notebook()

In [ ]:
# Sample data: 2D
df = px.data.stocks().set_index("date")
df.index = df.index.astype("datetime64[ns]")

stocks = xr.DataArray(
    df.values,
    dims=["date", "company"],
    coords={"date": df.index, "company": df.columns.tolist()},
    name="price",
)

# Sample data: 3D
df_gap = px.data.gapminder()
countries = ["United States", "China", "Germany", "Brazil"]
metrics = ["lifeExp", "gdpPercap"]

arrays = []
for metric in metrics:
    df_pivot = df_gap[df_gap["country"].isin(countries)].pivot(
        index="year", columns="country", values=metric
    )
    arrays.append(df_pivot.values)

data_3d = xr.DataArray(
    arrays,
    dims=["metric", "year", "country"],
    coords={
        "metric": metrics,
        "year": df_pivot.index.tolist(),
        "country": df_pivot.columns.tolist(),
    },
    name="value",
)

## Default Dimension Assignment

Dimensions are assigned to slots in order:

In [ ]:
# date → x, company → color
xpx(stocks).line()

## Explicit Assignment

Override the defaults by specifying which dimension goes where:

In [ ]:
# Swap: company → x, date → color
xpx(stocks.isel(date=[0, 50, 100])).bar(x="company", color="date")

## Skipping Slots with None

Use `None` to skip a slot, so dimensions shift to the next available slot:

In [ ]:
# Skip color → company goes to line_dash instead
xpx(stocks.sel(company=["GOOG", "AAPL", "MSFT"])).line(color=None)

## Available Slots

Different plot types have different slots:

| Plot | Slots (in order) |
|------|------------------|
| line | x, color, line_dash, facet_col, facet_row, animation_frame |
| scatter | x, color, symbol, facet_col, facet_row, animation_frame |
| bar | x, color, facet_col, facet_row, animation_frame |
| imshow | x, y, facet_col, facet_row, animation_frame |

In [ ]:
# line_dash for third dimension
xpx(stocks.sel(company=["GOOG", "AAPL"])).line(line_dash="company")

In [ ]:
# symbol for scatter
xpx(stocks.sel(company=["GOOG", "AAPL"])).scatter(symbol="company")

## Faceting

Create a grid of subplots with `facet_col` and `facet_row`:

In [ ]:
# One subplot per metric
xpx(data_3d).line(facet_col="metric")

In [ ]:
# Grid: metric x country
xpx(data_3d).line(x="year", facet_col="metric", facet_row="country")

## Animation

Animate over a dimension with `animation_frame`:

In [ ]:
# Animate through years
xpx(data_3d.sel(metric="lifeExp")).bar(x="country", animation_frame="year")

In [ ]:
# Animate heatmap
xpx(data_3d).imshow(animation_frame="metric")